In [1]:
# import zipfile
# with zipfile.ZipFile("dataset-resized.zip", 'r') as zip_ref:
#     zip_ref.extractall("trashnet")

In [2]:
from torchvision.models import resnet34,ResNet34_Weights
import torch.nn as nn
import torch
from torchsummary import summary
import torch.optim as optim
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Dataset loading

In [4]:
sample_img = Image.open("trashnet/dataset-resized/plastic/plastic1.jpg")
sample_img_arr = np.array(sample_img)
H,W,_ = np.shape(sample_img)

In [5]:
classes = ["cardboard","glass","metal","paper","plastic","trash"]
class_sizes = {"cardboard":403,"glass":501,"metal":410,"paper":594,"plastic":482,"trash":137}
class_nums =  {"cardboard":0,"glass":1,"metal":2,"paper":3,"plastic":4,"trash":5}

n_whole = np.sum(list(class_sizes.values()))

In [6]:
X = np.zeros((n_whole,H,W,3))
y = np.zeros((n_whole,))

In [7]:
n_whole

2527

In [8]:
#update X and y
total_count  = 0
for categ in classes:
    n_elem = class_sizes[categ]
    for i in range(1,1+n_elem):
        if total_count%300 == 0:
            print(f'total_count = {total_count}')
        X[total_count] = np.array(Image.open(f'trashnet/dataset-resized/{categ}/{categ}{i}.jpg'))
        y[total_count] = class_nums[categ]
        total_count += 1

total_count = 0
total_count = 300
total_count = 600
total_count = 900
total_count = 1200
total_count = 1500
total_count = 1800
total_count = 2100
total_count = 2400


In [9]:
X = X.reshape((n_whole,3,H,W))

In [29]:
train_X,test_X,train_y,test_y = train_test_split(X, y, test_size=500, random_state=42)
val_X,test_X , val_y,test_y = train_test_split(test_X, test_y, test_size=250, random_state=42)

In [31]:
# train_X = torch.LongTensor(train_X)
# train_y = torch.LongTensor(train_y)

In [11]:
train_X = torch.LongTensor(train_X)
train_y = torch.LongTensor(train_y)
train_dataset = torch.utils.data.TensorDataset(train_X, train_y)
trainloader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=64, shuffle=True, num_workers=2)

#not sure we would uuse validationloader or testloader
val_X = torch.LongTensor(val_X)
val_y = torch.LongTensor(val_y)
val_dataset = torch.utils.data.TensorDataset(val_X, val_y)
valloader = torch.utils.data.DataLoader(dataset=val_dataset, batch_size=64, shuffle=True, num_workers=2)

test_X = torch.LongTensor(test_X)
test_y = torch.LongTensor(test_y)
test_dataset = torch.utils.data.TensorDataset(test_X, test_y)
testloader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=64, shuffle=True, num_workers=2)

In [35]:
type(train_X.dtype)

torch.dtype

In [12]:
dataloaders = {'train':trainloader,'val':valloader,'test':testloader}

In [13]:
np.shape(train_X),np.shape(val_X),np.shape(test_X)

(torch.Size([2027, 3, 384, 512]),
 torch.Size([250, 3, 384, 512]),
 torch.Size([250, 3, 384, 512]))

## model initialization 

In [88]:
model = resnet34(weights = ResNet34_Weights.DEFAULT)
first_layer = list(model.children())[0]
print(f' first layer: {first_layer}')

 first layer: Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)


In [89]:
torch.save(model.state_dict(), "model_weights.pth")

In [85]:
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 6)

In [82]:
# freeze all layers
for param in model.parameters():
    param.requires_grad = False
last_layer = list(model.children())[-1]
print(f'except last layer: {last_layer}')
for param in last_layer.parameters():
    param.requires_grad = True

except last layer: Linear(in_features=512, out_features=6, bias=True)


In [83]:
model = model.to(device)
summary(model,(3,H,W))
# 512 x 384 x 3

Layer (type:depth-idx)                   Output Shape              Param #
├─Conv2d: 1-1                            [-1, 64, 192, 256]        (9,408)
├─BatchNorm2d: 1-2                       [-1, 64, 192, 256]        (128)
├─ReLU: 1-3                              [-1, 64, 192, 256]        --
├─MaxPool2d: 1-4                         [-1, 64, 96, 128]         --
├─Sequential: 1-5                        [-1, 64, 96, 128]         --
|    └─BasicBlock: 2-1                   [-1, 64, 96, 128]         --
|    |    └─Conv2d: 3-1                  [-1, 64, 96, 128]         (36,864)
|    |    └─BatchNorm2d: 3-2             [-1, 64, 96, 128]         (128)
|    |    └─ReLU: 3-3                    [-1, 64, 96, 128]         --
|    |    └─Conv2d: 3-4                  [-1, 64, 96, 128]         (36,864)
|    |    └─BatchNorm2d: 3-5             [-1, 64, 96, 128]         (128)
|    |    └─ReLU: 3-6                    [-1, 64, 96, 128]         --
|    └─BasicBlock: 2-2                   [-1, 64, 96, 128] 

Layer (type:depth-idx)                   Output Shape              Param #
├─Conv2d: 1-1                            [-1, 64, 192, 256]        (9,408)
├─BatchNorm2d: 1-2                       [-1, 64, 192, 256]        (128)
├─ReLU: 1-3                              [-1, 64, 192, 256]        --
├─MaxPool2d: 1-4                         [-1, 64, 96, 128]         --
├─Sequential: 1-5                        [-1, 64, 96, 128]         --
|    └─BasicBlock: 2-1                   [-1, 64, 96, 128]         --
|    |    └─Conv2d: 3-1                  [-1, 64, 96, 128]         (36,864)
|    |    └─BatchNorm2d: 3-2             [-1, 64, 96, 128]         (128)
|    |    └─ReLU: 3-3                    [-1, 64, 96, 128]         --
|    |    └─Conv2d: 3-4                  [-1, 64, 96, 128]         (36,864)
|    |    └─BatchNorm2d: 3-5             [-1, 64, 96, 128]         (128)
|    |    └─ReLU: 3-6                    [-1, 64, 96, 128]         --
|    └─BasicBlock: 2-2                   [-1, 64, 96, 128] 

In [17]:
criterion = nn.CrossEntropyLoss()
# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [90]:
for epoch in range(1):  # loop over the dataset multiple times

    running_loss = 0.0
    exp_lr_scheduler.step()
    model.train()
    for inputs, labels in dataloaders['train']:
        # get the inputs; data is a list of [inputs, labels]
        inputs = inputs.to(device)
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer_ft.zero_grad()
        # forward + backward + optimize
        # print(inputs[0]) 
        print(type(inputs.data))
        outputs = model(inputs)
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer_ft.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0
print('Finished Training')

<class 'torch.Tensor'>


RuntimeError: expected scalar type Long but found Float

In [18]:
# for epoch in range(1):  # loop over the dataset multiple times

#     running_loss = 0.0
#     exp_lr_scheduler.step()
#     for i, data in enumerate(dataloaders['train'], 0):
#         # get the inputs; data is a list of [inputs, labels]
#         inputs, labels = data

#         # zero the parameter gradients
#         optimizer_ft.zero_grad()
#         # forward + backward + optimize
#         outputs = model(inputs)
#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer_ft.step()

#         # print statistics
#         running_loss += loss.item()
#         if i % 2000 == 1999:    # print every 2000 mini-batches
#             print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
#             running_loss = 0.0

# print('Finished Training')

/Users/shiho/opt/anaconda3/envs/cos429/lib/python3.9/site-packages/torch/optim/lr_scheduler.py:139: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


RuntimeError: expected scalar type Long but found Float

## create dataset

In [12]:
arr = np.array([501,594,403,482,410,137])
200*arr/np.sum(arr)


array([39.65176098, 47.01226751, 31.89552829, 38.14800158, 32.44954491,
       10.84289672])